In [ ]:
import numpy as np 
import pandas as pd 
import seaborn as sns  
import plotly.express as px
import plotly.graph_objects as go
from statsmodels.graphics.mosaicplot import mosaic
import matplotlib.pyplot as plt
from itertools import *

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression
from statsmodels.formula.api import ols
import statsmodels.api as sm

import warnings
warnings.filterwarnings('ignore') 

In [ ]:
heart = pd.read_csv('../input/heart-disease-uci/heart.csv')

**sex** - пол                    
**cp** - тип боли в груди (4 values)              
**fbs** - уровень сахара в крови натощак> 120 мг / дл                  
**restecg** - результаты электрокардиографии в покое (значения 0,1,2)  
**exang**  - стенокардия, вызванная физической нагрузкой      
**slope** - наклон сегмента ST при пиковой нагрузке                
**ca** - количество крупных сосудов (0-3), окрашенных флурозопией                
**thal** - 3 = нормальный; 6 = исправленный дефект; 7 = обратимый дефект  

**age** - возраст                           
**trestbps** - артериальное давление в покое                 
**chol**  - холесторал сыворотки в мг / дл                 
**thalach** - maximum heart rate achieved - достигнута максимальная частота сердечных сокращений           
**oldpeak** - депрессия ST, вызванная упражнениями по сравнению с отдыхом                                      

**target** - Поле «цель» относится к наличию у пациента сердечного заболевания. Это целое число от 0 (отсутствие присутствия)

In [ ]:
print(heart.shape)
print(heart.info())
print(heart.isna().sum())
heart.head()

Some variable renaming for better visualization on graphs 

In [ ]:
heart['sex'] = heart['sex'].map({1 : "male", 0 : "female"})
heart['fbs>120'] = heart['fbs'].map({1 : "true", 0 : "false"}) # уровень сахара в крови больше ли 120
heart['exang'] = heart['exang'].map({1 : "yes", 0 : "no"}) # есть стенохардия или нет
del heart['fbs']

In [ ]:
heart.head()

In [ ]:
fig = go.Figure()

fig.add_trace(go.Bar(
    x=['0', '1'],
    y=list(heart.groupby('target').agg({'target': 'count'}).target)
))
fig.update_layout(
    width=500,
    height=500,
    yaxis=dict(
        title_text="target"))

In [ ]:
fig, axs = plt.subplots(2, 4, figsize=(20, 10), sharey=False)
dimensions=['sex', 'cp', 'fbs>120', 'restecg', 'exang', 'slope', 'ca', 'thal']

f = 0
for r in range(2):
    for c in range(4):
        ax = axs[r, c]
        ax = sns.histplot(data=heart, x="target", hue='{}'.format(
            dimensions[f]), shrink=5, multiple="dodge", ax=ax)
        f+=1
        
plt.show()        
plt.close()

In [ ]:
fig = px.parallel_categories(heart, dimensions, color='target')
fig.show()

In [ ]:
fig, axs = plt.subplots(5, 2, figsize=(20, 30), sharey=False)
h = ['age', 'trestbps', 'chol', 'thalach', 'oldpeak']

for r in range(5):
    for c in range(1):
        ax = axs[r, c]
        ax = sns.histplot(heart['{}'.format(h[r])], ax=ax, kde=True)
    for c in range(1,2): 
        ax = axs[r, c]
        ax = sns.boxplot(heart['{}'.format(h[r])], ax=ax)
        
plt.show()        
plt.close()

In [ ]:
fig, axs = plt.subplots(8, 3, figsize=(17, 40), sharey=False)

for r in range(8):
    for c in range(1):
        ax = axs[r, c]
        ax = sns.histplot(heart['{}'.format(dimensions[r])], ax=ax)
        
    for c in range(1,2): 
        ax = axs[r, c]
        ax = mosaic(heart, ['{}'.format(dimensions[r]),"target"], gap=0.01, ax=ax) 
    
    for c in range(2,3): 
        ax = axs[r, c]
        ax = sns.barplot('{}'.format(dimensions[r]), 'target', data=heart, ax=ax)
        
plt.show() 

1. По первой строчке графиков можно сказать что больше в данных представленно мужчинами,при этом кажется что шанс НЕ получить сердечное заболевание больше у мужчин, чем у женщин (однако это может быть обусловленно просто большим кол-вом мужчин в выборке). Так же у мужчин распределение получивших сердчное заболевание и не получивших его римерно равномерное, у женщин напротив есть перекос в сторону имевших заолевание         

    According to the first line of the graphs, we can say that more in the data are presented by men, while it seems that the chance of NOT getting heart disease is greater in men than in women (however, this may simply be due to the large number of men in the sample). Also, in men, the distribution of those who received heart disease and did not receive it is approximately uniform, in women, on the contrary, there is a bias towards those who had a disease 

2. Тип боли в груди (2я строчка) - люди с типом боли в груди (cp) - 0 имеют самые малые шансы на получения сердечного заболевания - скорее всего тип боли 0 означает вообще без боли. типы боли 1,2,3 имеют достаточно высокие шансы на заболевание. При этом вероятность получить заболевание выще в 1 типе боли в груди

    Type of chest pain (2nd line) - people with type of chest pain (cp) - 0 have the least chance of getting heart disease - most likely type of pain 0 means no pain at all. types of pain 1,2,3 have a fairly high chance of getting sick. At the same time, the likelihood of getting the disease is higher in type 1 of chest pain 

3. уровень сахара в крови натощак (3я строчка) - шансы на сердечное заболевание будут примерно одинаковыми для людей имеющих сахар в крови>120 и меньше 120

    fasting blood sugar (3rd line) - the chances of heart disease will be about the same for people with blood sugar> 120 and less than 120 

4. restecg - результаты электрокардиографии в покое (4я строчка) - для значения электрокардиографии 1 шансы на НЕПОЛУЧЕНИЯ заболевания будут максимальными, у нас так же есть значения кардиограммы 2, и на самом деле максимальные шансы не получить заболевания именно там, но мне кажется что там настолько мало наблюдений,что учитывать их или нет стоит под вопросом

    restecg - the results of electrocardiography at rest (4th line) - for the value of electrocardiography 1, the chances of NOT getting the disease will be maximum, we also have the values of the cardiogram 2, and in fact, the maximum chances of not getting the disease are there, but it seems to me that there are so few observations, what to take into account them or not is questionable 

5. exang - стенокардия, вследствии физической нагрузки (5я строчка), во первых по правой гистограмме у нас видно что в данных содержится больше людей без стенохардии. Так же видно что вероятность получить болезнь сердца будет выше у людей без стенокардии (однако это может быть обусловленно просто большим кол-вом людей без стенохардии)

    exang - angina pectoris due to physical activity (5th line), firstly, according to the right histogram, we can see that the data contains more people without angina pectoris. It can also be seen that the likelihood of getting heart disease will be higher in people without angina pectoris (however, this may simply be due to a large number of people without angina pectoris) 

6. slope - наклон сегмента ST при пиковой нагрузке, вероятность заболеть или получить здоровое сердце будет примерно одинаковым при наклоне сегмента 0. Вероятность здорового средца у человека при наклоне сегмента 1 будет наибольшей

    slope - the slope of the ST segment at peak load, the likelihood of getting sick or getting a healthy heart will be approximately the same when segment 0 is tilted. The likelihood of a healthy heart in a person when segment 1 is tilted will be the greatest 

7. ca - количество крупных сосудов (0-3), окрашенных флурозопией - наибольшее кол-во людей имеют минимальное (0) кол-во крупных сосудов, они же имеют минимальную вероятность получить болезнь сердца 

    ca - the number of large vessels (0-3) stained with flurosopia - the largest number of people have a minimum (0) number of large vessels, they also have the lowest probability of getting heart disease 

8. thal - минимальная вероятность получить болезнь сердца, у людей имеющих thal 1 и 3  

    thal - the minimum chance of getting heart disease in people with thal 1 and 3 

In [ ]:
abcd = ['age','trestbps','chol','thalach','oldpeak']
f = [i for i in combinations(abcd, 2)]

In [ ]:
f

In [ ]:
fig, axs = plt.subplots(10, 8, figsize=(25, 60), sharey=False) 

for k in range(len(f)):
    for c in range(1):
        ax = axs[k,c]
        ax = sns.scatterplot(data=heart, x=f[k][0], y=f[k][1], hue="sex", ax=ax)
       
    for c in range(1,2):
        ax = axs[k,c]
        ax = sns.scatterplot(data=heart, x=f[k][0], y=f[k][1], hue="cp", ax=ax)

    for c in range(2,3):
        ax = axs[k,c]
        ax = sns.scatterplot(data=heart, x=f[k][0], y=f[k][1], hue="fbs>120", ax=ax)
        
    for c in range(3,4):
        ax = axs[k,c]
        ax = sns.scatterplot(data=heart, x=f[k][0], y=f[k][1], hue="restecg", ax=ax)
        
    for c in range(4,5):
        ax = axs[k,c]
        ax = sns.scatterplot(data=heart, x=f[k][0], y=f[k][1], hue="exang", ax=ax)
        
    for c in range(5,6): 
        ax = axs[k,c]
        ax = sns.scatterplot(data=heart, x=f[k][0], y=f[k][1], hue="slope", ax=ax)
        
    for c in range(6,7):
        ax = axs[k,c]
        ax = sns.scatterplot(data=heart, x=f[k][0], y=f[k][1], hue="ca", ax=ax)
        
    for c in range(7,8):
        ax = axs[k,c]
        ax = sns.scatterplot(data=heart, x=f[k][0], y=f[k][1], hue="thal", ax=ax)
    
plt.show()  

In [ ]:
fig = px.scatter(heart, x="age", y='trestbps', animation_frame="cp", color="sex")
fig.show()

In [ ]:
fig = px.scatter(heart, x="age", y='thalach', animation_frame="cp", color="sex")
fig.show()

In [ ]:
fig = px.scatter(heart, x="age", y='chol', animation_frame="cp", color="sex")
fig.show()

In [ ]:
heart['sex'] = heart['sex'].map({'male': 1, 'female': 0})
heart['fbs>120'] = heart['fbs>120'].map({"true": 1, "false": 0}) 
heart['exang'] = heart['exang'].map({"yes": 1, "no": 0})

target = heart['target']
heart = heart.drop(['target'], axis=1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(heart, target, 
                                                    test_size=0.33, random_state=42)

Variable standardization pipeline 

In [ ]:
class OldDataFrameSelector(BaseEstimator, TransformerMixin):
    
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return X[self.attribute_names].values

In [ ]:
abcd_pipeline_scaller = Pipeline([
        ('selector', OldDataFrameSelector(abcd)),
        ("scalermm", StandardScaler())   
    ])

dimensions_pipeline_scaller = Pipeline([
        ('selector', OldDataFrameSelector(dimensions))  
    ])

full_pipeline = FeatureUnion(transformer_list=[
    ("abcd_pipeline_scaller", abcd_pipeline_scaller),
    ("dimensions_pipeline_scaller", dimensions_pipeline_scaller),
    ])

In [ ]:
X_train = pd.DataFrame(full_pipeline.fit_transform(X_train), columns=abcd + dimensions)   
X_test = pd.DataFrame(full_pipeline.fit_transform(X_test), columns=abcd + dimensions)   

Let's take a standard model in which there will be only one nominative variable (gender) and we will add other variables to it one by one.
At each stage, we will observe how the level of significance of the variable changes. The null hypothesis there is no relationship between the variables and the addition of any variable will not affect the significance of the variables already added. 

Возмем стандартную модель в которой будет только одна номинативная переменная (пол) и почередно будем добавлять в нее другие переменные.
На каждом этапе будем наблюдать как меняется уровень значимости переменной. Нулевая гипотеза между переменными нет никакой взаимосвязи и добавление какой либо переменной не окажет влияния на значимость уже добавленных переменных

In [ ]:
X_train['target'] = y_train.values
X_test['target'] = y_test.values
X_train.rename(columns = {'fbs>120':'fbs'}, inplace = True) 
X_test.rename(columns = {'fbs>120':'fbs'}, inplace = True) 

In [ ]:
X_train.head(5)

In [ ]:
X_test.head()

In [ ]:
test = 'target ~'
for i in range(len(dimensions)):
    if i == 0: test = test + 'C({})'.format(dimensions[i])
    else: test = test + '+C({})'.format(dimensions[i])
    print(test)
    model = ols(test, data=X_train).fit()
    aov_table1 = sm.stats.anova_lm(model, typ=2)
    print(aov_table1)
    print('-----------------------------------------------')

In [ ]:
#https://www.kaggle.com/ztyh0121/lab1-introduction-regression-anova 
#https://www.statsmodels.org/stable/anova.html?highlight=anova_lm

In [ ]:
moore = sm.datasets.get_rdataset("Moore", "carData", cache=True) # load
data = moore.data
data = data.rename(columns={"partner.status": "partner_status"}) # make name pythonic

In [ ]:
moore_lm = ols('conformity ~ C(fcategory, Sum)*C(partner_status, Sum)', data=data).fit()
table = sm.stats.anova_lm(moore_lm, typ=2) # Type 2 Anova DataFrame
print(table)